# Import libs

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import yaml
import pprint

if '../../../notebooks/' not in sys.path:
    sys.path.append('../../../notebooks/')

import utils.constants as cts

from data_loaders.data_loader import DLName
from gt_loaders.gt_names import GTName
from exp_runner import ExperimentRunner
from model_trainer import BaseModel, Optimizer

# create_config Function

In [3]:
def create_config(req, ds, aligned):
    return { 
                'use_neptune': False,
                'exp_params' : {
                    'name': 'train_vgg16',
                    'description': f'Training network for {req.value.upper()} requisite',
                    'tags': ['vgg16', 'ground truths', 'adamax', ds.value.lower(), 'binary_output', req.value.lower()],
                    'src_files': ['exp_runner.py', 'data_processor.py', 'model_trainer.py', 'model_evaluator.py']
                },
                'properties': {
                    'reqs': [req],
                    'aligned': aligned,
                    'use_gt_data': True,
                    'gt_names': {
                        'train_validation': [],
                        'test': [],
                        'train_validation_test': [ds]
                    },
                    'balance_input_data': False,
                    'train_model': True,
                    'save_trained_model': True,
                    'model_name': '',
                    'sample_training_data': False,
                    'sample_prop': 1.
                },
                'net_train_params': {
                    'base_model': BaseModel.VGG16,
                    'batch_size': 64,
                    'n_epochs': 2,
                    'early_stopping': 10,
                    'learning_rate': 1e-3,
                    'optimizer': Optimizer.ADAMAX,
                    'train_prop': 0.9,
                    'validation_prop': 0.05,
                    'test_prop': 0.05,
                    'validation_split': 0.1,
                    'dropout': 0.3
                }
            }

# Execute Experiment Runner

In [4]:
reqs_list = [cts.ICAO_REQ.DARK_GLASSES, cts.ICAO_REQ.MOUTH]
ds_list = [GTName.FVC]
align_list = [True]

for req in reqs_list:
    for ds in ds_list:
        for alig in align_list:
            exp_cf = create_config(req, ds, alig)
            with open('exp_config.yaml','w') as f:
                yaml.dump([exp_cf], f)
                !python exp_runner.py -c 'exp_config.yaml'



Loading experiment config from exp_config.yaml
..Experiment configs loaded with success!
-----
Use Neptune:  False
-----
Args: 
{'exp_params': {'description': 'Training network for DARK_GLASSES requisite',
                'name': 'train_vgg16',
                'src_files': ['exp_runner.py',
                              'data_processor.py',
                              'model_trainer.py',
                              'model_evaluator.py'],
                'tags': ['vgg16',
                         'ground truths',
                         'adamax',
                         'fvc',
                         'binary_output',
                         'dark_glasses']},
 'net_train_params': {'base_model': <BaseModel.VGG16: {'target_size': (224, 224), 'prep_function': <function preprocess_input at 0x7f47366a2280>}>,
                      'batch_size': 64,
                      'dropout': 0.3,
                      'early_stopping': 10,
                      'learning_rate': 0.001,
          